In [ ]:
!pip install asana

In [ ]:
import asana
from asana.rest import ApiException
from pprint import pprint
import requests
import pandas as pd
from pandas import json_normalize
import json

In [ ]:
pd.set_option('display.max_colwidth', None)

In [ ]:
# Configure OAuth2 access token for authorization
configuration = asana.Configuration()
configuration.access_token = 'xxxxx' #troque pelo seu acesso
api_client = asana.ApiClient(configuration)

api_instance = asana.TasksApi(api_client)

# ID do projeto (substitua pelo ID do projeto desejado)
project_id = 'xxxxxx' #id projeto Piauí

In [ ]:
# Define os campos que deseja incluir na consulta
opt_fields = [
    "gid",
    "created_at",
    "completed_at",
    "modified_at",
    "name",
    "memberships.section.name",
    "assignee.name",
    "assignee.email",
    "start_on",
    "due_on",
    "tags.name",
    "notes",
    "projects.name",
]

def get_all_tasks_and_subtasks(task, parent_task_name=None):
    task_data.append({
        'section_name': task.memberships[0].section.name if task.memberships else None,
        'task_name': parent_task_name if parent_task_name else task.name,
        'subtask_name': task.name,
        'assignee': task.assignee.name if task.assignee else None,
        'subtask_notes': task.notes,
        'due_on': task.due_on.strftime("%d/%m/%Y") if task.due_on else None,
        'tag_names': [tag.name for tag in task.tags] if task.tags else None
    })

    subtask_response = api_instance.get_subtasks_for_task(task.gid, opt_fields=opt_fields)
    subtasks = subtask_response.data

    for subtask in subtasks:
        get_all_tasks_and_subtasks(subtask, parent_task_name=parent_task_name if parent_task_name else task.name)


# Obtendo as tarefas do projeto
try:
    project_tasks_response = api_instance.get_tasks_for_project(project_id, opt_fields=opt_fields)
    project_tasks = project_tasks_response.data
except Exception as e:
    print("Erro ao obter tarefas do projeto:", str(e))
    exit()

# Lista para armazenar os dados de tarefas e subtarefas
task_data = []

# Lista para armazenar os dados de tarefas e subtarefas
task_data = []

# Iterar sobre as tarefas do projeto
for task in project_tasks:
    get_all_tasks_and_subtasks(task)

# Crie um DataFrame a partir dos dados coletados
df = pd.DataFrame(task_data)

# Dicionário de mapeamento para tradução de nomes de colunas
nome_coluna_map = {
    'section_name' : 'Seção',
    'task_name': 'Nome da Tarefa',
    'subtask_name': 'Nome da Subtarefa',
    'assignee': 'Responsável',
    'subtask_notes': 'Notas',
    'due_on': 'Concluída em',
    'tag_names': 'Tags',
}

# Iterar sobre as chaves do dicionário de mapeamento
for coluna_em_ingles, coluna_em_portugues in nome_coluna_map.items():
    # Verifique se a coluna em inglês existe no DataFrame
    if coluna_em_ingles in df.columns:
        # Renomeie a coluna no DataFrame
        df = df.rename(columns={coluna_em_ingles: coluna_em_portugues})


Dados apresentados no relátorio

In [ ]:
# Filtrar as tarefas pela tag 'relatório executivo'
filtro_tag_relatorio = df['Tags'].apply(lambda x: x is not None and 'relatório executivo' in x)

# Filtrar as tarefas pelo mês e ano de conclusão
filtro_mes = df['Concluída em'].str.contains("/09/2023")

# Combinar os dois filtros
tarefas_relatorio = df[filtro_tag_relatorio & filtro_mes]

tarefas_relatorio = tarefas_relatorio[["Seção", "Nome da Tarefa",	"Nome da Subtarefa",	"Responsável",	"Notas",	"Concluída em",	"Tags"]]

# Salvar o DataFrame em um arquivo CSV
nome_arquivo_csv = 'tarefaspiaui.csv'
df.to_csv(nome_arquivo_csv, index=False)

tarefas_relatorio


In [ ]:
# Contar o número de tarefas em tarefas_relatorio
numero_de_tarefas = tarefas_relatorio.shape[0]

# Mostrar o resultado
print(f"Número de tarefas: {numero_de_tarefas}")